# Initialize

## Imports

In [1]:
import sys; module_name = '..\src'
if module_name not in sys.path: sys.path.insert(0, module_name)

import pandas as pd
import numpy as np
from pathlib import Path
from pprint import pprint
from jinja2 import Environment, FileSystemLoader

%load_ext autoreload

In [2]:
%autoreload 2
from graph_db_util import read_cypher_query

## Configure

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [10]:
# jinja settings
environment = Environment(loader=FileSystemLoader("../src/templates/"))

## Define

# Generate a TOC for the domain

## Get the data

In [11]:
domain_name = 'Finance'

# get all business processes owned directly by finance
qry = f"""MATCH (:Domain {{name:'{domain_name}'}})-[:OWNS]-(bp:BusinessProcess)
RETURN bp.name as name"""
df = read_cypher_query(qry)
bp1 = set(df['name'])

# get all business processes owned by any sub-domain of finance
qry = f"""MATCH (:Domain {{name:'{domain_name}'}})-[:PARENT*1..]-(d)
MATCH (d)-[:OWNS]-(bp:BusinessProcess)
RETURN bp.name as name""" # TODO test path depth > 1
df = read_cypher_query(qry)
bp2 = set(df['name'])
business_processes = set.union(bp1, bp2)
business_processes

{'AUM', 'Management Fees'}

In [12]:
qry = f"""MATCH (p:Domain {{name:'{domain_name}'}})-[:PARENT]-(c:Domain) RETURN c.name AS name"""
df = read_cypher_query(qry)
data_domains = set(df['name'])
data_domains

{'AUM',
 'Cash Flows',
 'Commission Payments',
 'Customers',
 'Expenses',
 'FX Rates',
 'Revenue'}

In [13]:
qry = f"""MATCH (p:Domain)-[:OWNS]-(c:System) RETURN c.name AS name"""
df = read_cypher_query(qry)
systems = set(df['name'])

qry = f"""MATCH (p:Domain)-[:OWNS]-(c:App) RETURN c.name AS name"""
df = read_cypher_query(qry)
applications = set(df['name'])
applications

{'Commissions App',
 'Management Fees App',
 'Revport Admin App',
 'State Tax App'}

## Generate the markdown

In [14]:
template = environment.get_template("domain.j2")
content = template.render(domain_name=domain_name
                          ,business_processes=business_processes
                          ,systems=systems
                          ,data_domains=data_domains
                          ,applications=applications)

print(content)

# Finance TOC

## Business Processes

- [Management Fees](/Management-Fees.md)
- [AUM](/AUM.md)

## Systems

- [Revport](/Revport.md)
- [TM1](/TM1.md)

## Applications

- [Commissions App](/Commissions-App.md)
- [Management Fees App](/Management-Fees-App.md)
- [Revport Admin App](/Revport-Admin-App.md)
- [State Tax App](/State-Tax-App.md)

## Data

- [Commission Payments](/Commission-Payments.md)
- [AUM](/AUM.md)
- [Revenue](/Revenue.md)
- [Expenses](/Expenses.md)
- [FX Rates](/FX-Rates.md)
- [Customers](/Customers.md)
- [Cash Flows](/Cash-Flows.md)


## Publish the markdown

In [ ]:
md_path = Path(r"..\markdown")
file_path = md_path / f'{domain_name}-Domain.md'

with open(file_path, mode='w') as f:
    f.write(content)

# Generate Markdown for Business Processes